# Migrate db from `perch-hoplite < 1.0`

If you've previously used `perch-hoplite < 1.0` to create a database, use this notebook to convert it to `perch-hoplite == 1.*`.

In [ ]:
# @title Imports

import json
import shutil
import sqlite3
from typing import Any

from etils import epath
from IPython.display import display
import ipywidgets as widgets
from ml_collections import config_dict
import numpy as np
import tqdm

from perch_hoplite.db import interface
from perch_hoplite.db import sqlite_usearch_impl

In [ ]:
# @title Inputs and outputs {vertical-output: true}

# @markdown Input db location:
input_db_path = ""  # @param {type: "string"}
input_db_path = epath.Path(input_db_path)

# @markdown Output db location:
output_db_path = ""  # @param {type: "string"}
output_db_path = epath.Path(output_db_path)

# @markdown Window size (in seconds) used by the embeddings model.
# @markdown Needed to compute the end offset because the old database stored only the start offset.
window_size_s = 5.0  # @param {type: "number"}

In [ ]:
# @title Delete old db migration {vertical-output: true}

if output_db_path.exists():
  # Ask the user to confirm if they want to delete the old output directory.

  # 1. Setup the widgets.
  label = widgets.HTML(value=f"Output directory already exists: <b>{output_db_path}</b>")
  btn_delete = widgets.Button(description="Delete Directory", button_style="warning", icon="trash")
  btn_confirm = widgets.Button(description="Yes", button_style="danger", layout={"display": "none"})
  btn_cancel = widgets.Button(description="Cancel", layout={"display": "none"})
  output = widgets.Output()

  # 2. Define the logic.
  def show_confirmation(b):
      btn_delete.layout.display = "none"
      btn_confirm.layout.display = "inline-block"
      btn_cancel.layout.display = "inline-block"
      with output:
          output.clear_output()
          print("⚠️ Are you sure? This cannot be undone.")

  def cancel_action(b):
      btn_delete.layout.display = "inline-block"
      btn_confirm.layout.display = "none"
      btn_cancel.layout.display = "none"
      with output:
          output.clear_output()

  def perform_deletion(b):
      with output:
          output.clear_output()
          try:
              shutil.rmtree(output_db_path)
              print(f"✅ Successfully deleted: {output_db_path}")
              btn_confirm.disabled = True
              btn_cancel.description = "Done"
          except Exception as e:
              print(f"❌ Error: {e}")

  # 3. Wire up the buttons.
  btn_delete.on_click(show_confirmation)
  btn_cancel.on_click(cancel_action)
  btn_confirm.on_click(perform_deletion)

  # 4. Display the UI.
  controls = widgets.HBox([btn_delete, btn_confirm, btn_cancel])
  display(widgets.VBox([label, controls, output]))

else:

  label = widgets.HTML(value=f"Output directory doesn't exist yet: <b>{output_db_path}</b>")
  display(label)

In [ ]:
# @title Convert the SQLite database

# Define helper function.
def deserialize_array(serialized: bytes, dtype: type[Any]) -> np.ndarray:
  return np.frombuffer(serialized, dtype=np.dtype(dtype).newbyteorder('<'))

# Check the input database to make sure it's in the old format.
old_db = sqlite3.connect(input_db_path / "hoplite.sqlite")
old_cursor = old_db.cursor()
old_cursor.execute("""
  SELECT name
  FROM sqlite_master
  WHERE type = "table"
""")
old_tables = {row[0] for row in old_cursor.fetchall()}
expected_old_tables = {"hoplite_metadata", "hoplite_sources", "hoplite_embeddings", "hoplite_labels"}
assert len(old_tables & expected_old_tables) == 4, "Something is wrong with the old database. Please check its tables."

# Load the usearch_cfg.
old_cursor.execute("""
  SELECT data
  FROM hoplite_metadata
  WHERE key = "usearch_config"
""")
usearch_cfg = config_dict.ConfigDict(json.loads(old_cursor.fetchone()[0]))

# Create the new database.
new_db = sqlite_usearch_impl.SQLiteUSearchDB.create(output_db_path, usearch_cfg)

# Copy the hoplite metadata.
old_cursor.execute("""
  SELECT key, data
  FROM hoplite_metadata
  WHERE key != "usearch_config"
""")
for row in old_cursor.fetchall():
  key = row[0]
  value = config_dict.ConfigDict(json.loads(row[1]))
  new_db.insert_metadata(key, value)
new_db.commit()

print("Copying old sources into new deployments and recordings...")
old_dataset_to_new_deployment_id = {}
old_source_id_to_new_recording_id = {}
old_cursor.execute("""
  SELECT DISTINCT dataset
  FROM hoplite_sources
""")
for dataset, in tqdm.tqdm(old_cursor.fetchall()):
  old_dataset_to_new_deployment_id[dataset] = new_db.insert_deployment(
      name=dataset,
      project=dataset,
  )
old_cursor.execute("""
  SELECT id, dataset, source
  FROM hoplite_sources
""")
for source_id, dataset, source in tqdm.tqdm(old_cursor.fetchall()):
  old_source_id_to_new_recording_id[source_id] = new_db.insert_recording(
      filename=source,
      deployment_id=old_dataset_to_new_deployment_id[dataset],
  )
new_db.commit()

print("Creating windows...")
# Copy the old embeddings into new windows. Do not reinsert the embedding vector into USearch since
# we're reusing the vector search index as it is. The trick for obtaining the same window ids is to
# insert windows in the original order and remove gaps.
old_cursor.execute("""
  SELECT id, source_idx, offsets
  FROM hoplite_embeddings
  ORDER BY id
""")
last_window_id = 0

for embedding_id, source_id, old_offsets in tqdm.tqdm(old_cursor.fetchall()):
  while last_window_id + 1 < embedding_id:
    dummy_window_id = new_db.insert_window(
        recording_id=1,
        offsets=[],
        embedding=None,
    )
    new_db.remove_window(dummy_window_id)
    last_window_id += 1

  # The old database stored only the start offset. Use the provided `window_size_s` to compute the
  # end offset.
  start_offset = deserialize_array(old_offsets, np.float32)[0].item()
  end_offset = start_offset + window_size_s
  window_id = new_db.insert_window(
      recording_id=old_source_id_to_new_recording_id[source_id],
      offsets=[start_offset, end_offset],
      embedding=None,
  )
  assert window_id == embedding_id

  last_window_id += 1

new_db.commit()

# Map window_id to new (recording_id, offsets) tuples, to be used for mapping annotations in the
# new table schema.
recording_offsets = {
    window.id : (window.recording_id, window.offsets)
    for window in new_db.get_all_windows()
}

# Copy the old labels into new annotations. No need to map the window ids since they remain the same.
old_cursor.execute("""
  SELECT embedding_id, label, type, provenance
  FROM hoplite_labels
""")
for old_embedding_id, old_label, old_type, old_provenance in tqdm.tqdm(old_cursor.fetchall()):
  new_db.insert_annotation(
      recording_id=recording_offsets[old_embedding_id][0],
      offsets=recording_offsets[old_embedding_id][1],
      label=old_label,
      label_type=interface.LabelType(old_type),
      provenance=old_provenance,
  )
new_db.commit()

In [ ]:
# @title Inspect and close Hoplite db {vertical-output: true}

print("Deployments:", len(new_db.get_all_deployments()))
print("Recordings:", len(new_db.get_all_recordings()))
print("Windows:", len(new_db.get_all_windows()))
print("Annotations:", len(new_db.get_all_annotations()))
print("Embeddings:", new_db.count_embeddings())

del new_db

In [ ]:
# @title Copy the vector search index {vertical-output: true}

output_index_path = (input_db_path / "usearch.index").copy(
    output_db_path / "usearch.index",
    overwrite=True,
)
print("Output index:", output_index_path)

In [ ]:
# @title Reload and reinspect Hoplite db {vertical-output: true}

new_db = sqlite_usearch_impl.SQLiteUSearchDB.create(output_db_path, usearch_cfg)

print("Deployments:", len(new_db.get_all_deployments()))
print("Recordings:", len(new_db.get_all_recordings()))
print("Windows:", len(new_db.get_all_windows()))
print("Annotations:", len(new_db.get_all_annotations()))
print("Embeddings:", new_db.count_embeddings())

In [ ]:
# @title Inspect old SQLite database for final comparison {vertical-output: true}

num_datasets = old_cursor.execute("""
  SELECT COUNT(DISTINCT dataset)
  FROM hoplite_sources
""").fetchone()[0]
num_sources = old_cursor.execute("""
  SELECT COUNT(*)
  FROM hoplite_sources
""").fetchone()[0]
num_embeddings = old_cursor.execute("""
  SELECT COUNT(*)
  FROM hoplite_embeddings
""").fetchone()[0]
num_labels = old_cursor.execute("""
  SELECT COUNT(*)
  FROM hoplite_labels
""").fetchone()[0]

print("Datasets:", num_datasets)
print("Sources:", num_sources)
print("Embeddings:", num_embeddings)
print("Labels:", num_labels)